<a href="https://colab.research.google.com/github/computerphotophytoart/OMCBetaTest/blob/main/OysterMusselClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastai
from fastai.vision.all import * #This imports fastai library, -Uqq is used to silence output msgs

In [ ]:
foodpath = untar_data(URLs.FOOD)

In [ ]:
len(get_image_files(foodpath))

In [ ]:
pd.read_json('/root/.fastai/data/food-101/test.json')

In [ ]:
labelA = 'oysters'
labelB = 'mussels'

In [ ]:
for img in get_image_files(foodpath):
  #Check if labelA is in file name
  if labelA in str(img):
    img.rename(f"{img.parent}/{labelA}-{img.name}")
  elif labelB in str(img):
    img.rename(f"{img.parent}/{labelB}-{img.name}")
  else: os.remove(img)
#If labelA or labelB aren't found, remove the files
len(get_image_files(foodpath)) #Get number of images in directory

**Model Training**

In [ ]:
def GetLabel(fileName):
  return fileName.split('-')[0]

GetLabel("oysters-987252.jpg") #takes file name of an image as input. if the file name is 'oysters-987252.jpg', it extracts the label 'oysters' and returns it."

In [ ]:
dls = ImageDataLoaders.from_name_func( #creates data loaders to retrieve files
    foodpath, get_image_files(foodpath), valid_pct=0.2, seed=420, #retrieve files, takes only 20% with seed
    label_func=GetLabel, item_tfms=Resize(32)) #resizes images to 32x32

dls.train.show_batch() #display images from training set

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate, pretrained=True) #change resnet34 to resnet50 for faster results but longer training time
learn.fine_tune(epochs=1) #more epochs = higher iterations so better results but longer waiting times. keep between 1-3.

**Model Testing**

In [ ]:
from google.colab import files
uploader = files.upload()

for img in uploader.items():
  uploadedImg = img[0]

img = PILImage.create(uploadedImg)
img.show()

label,_,probs = learn.predict(img)

print(f"This is a {label}.")
print(f"{labelA} {probs[1].item():.6f}")
print(f"{labelB} {probs[0].item():.6f}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

for i in range(0,10):
  #Load random image
  randomIndex = random.randint(0, len(get_image_files(foodpath))-1)
  img = mpimg.imread(get_image_files(foodpath)[randomIndex])
  #Put into Model
  label,_,probs = learn.predict(img)

  #Create Figure using Matplotlib
  fig = plt.figure()
  ax = fig.add_subplot() #Add Subplot (For multiple images)
  imgplot = plt.imshow(img) #Add Image into Plot
  ax.set_title(label) #Set Headline to predicted label

  #Hide numbers on axes
  plt.gca().axes.get_yaxis().set_visible(False)
  plt.gca().axes.get_xaxis().set_visible(False)

In [ ]:
#Confusion Matrix
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
interp.plot_top_losses(6)

**Download Model**

In [ ]:
learn.export()

In [ ]:
modelPath = get_files(foodpath, '.pkl')[0]
modelPath

In [ ]:
learn_inf = load_learner(modelPath)
learn_inf.predict(mpimg.imread(get_image_files(foodpath)[0]))

In [ ]:
learn_inf.dls.vocab

In [ ]:
from google.colab import files
files.download(modelPath)